In [13]:
#-*- coding: utf-8 -*-
import requests
import json, os
url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")

resp = requests.put('http://' + url + '/api/v1/type/service/chatbot/2222/', 
                    json={  "unique_id" : "1111",
                            "package_id" : "abcd",
                            "intent_id" : "",
                            "intent_name" : "",
                            "input_data" : "김승우 과장 휴가",
                            "convert_data" : "",
                            "intent_history" : [],
                            "request_type" : "text",
                            "service_type" : "",
                            "story_board_id" : "", 
                            "story_req_entity" : {}, 
                            "story_set_entity" : {},
                            "opt_sel_list" : {},
                            "ontology_id" : "", 
                            "ontology_req_parms" : {}, 
                            "ontology_set_parms" : {},
                            "output_data" : ""
                          })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {"output_data": "", "story_board_id": "", "unique_id": "1111", "intent_history": [], "opt_sel_list": {}, "story_req_entity": {}, "service_type": "", "intent_id": "6", "ontology_set_parms": {}, "package_id": "abcd", "ontology_id": "", "story_set_entity": {"이름": "김승우", "근태코드": "휴가", "직급": "과장"}, "convert_data": " [이름] [직급] [근태코드]", "request_type": "text", "input_data": "김승우 과장 휴가", "ontology_req_parms": {}, "intent_name": ""}
